# Bước 1: Download dữ liệu

In [ ]:
! curl -O https://cogcomp.seas.upenn.edu/Data/QA/QC/TREC_10.label
! curl -O https://cogcomp.seas.upenn.edu/Data/QA/QC/train_5500.label

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 23354  100 23354    0     0   4480      0  0:00:14  0:00:04  0:00:10  1618  0  0:00:05  0:00:05 --:--:--  5262
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  327k  100  327k    0     0  36567      0  0:00:09  0:00:09 --:--:-- 26389


In [ ]:
! wc -l TREC_10.label
# ! head -10 TREC_10.label


! wc -l train_5500.label
! head -10 train_5500.label
# ! cp train_5500.label  train_5500.txt

     500 TREC_10.label
    5452 train_5500.label
DESC:manner How did serfdom develop in and then leave Russia ?
ENTY:cremat What films featured the character Popeye Doyle ?
DESC:manner How can I find a list of celebrities ' real names ?
ENTY:animal What fowl grabs the spotlight after the Chinese Year of the Monkey ?
ABBR:exp What is the full form of .com ?
HUM:ind What contemptible scoundrel stole the cork from my lunch ?
HUM:gr What team did baseball 's St. Louis Browns become ?
HUM:title What is the oldest profession ?
DESC:def What are liver enzymes ?
HUM:ind Name the scar-faced bounty hunter of The Old West .


# Bước 2: Đọc dữ liệu

In [ ]:
import codecs
def _generate_examples(filepath):
    examples = []
    with codecs.open(filepath, "rb") as f:
        for id_, row in enumerate(f):
            # One non-ASCII byte: sisterBADBYTEcity. We replace it with a space
            label, _, text = row.replace(b"\xf0",
                                         b" ").strip().decode().partition(" ")
            coarse_label, _, fine_label = label.partition(":")
            examples.append((id_, {
                "label-coarse": coarse_label,
                "label-fine": fine_label,
                "text": text,
            }))
    return examples

In [ ]:
train = _generate_examples("train_5500.label")
test = _generate_examples("TREC_10.label")

In [ ]:
print(len(train))
print(train[0])

5452
(0, {'label-coarse': 'DESC', 'label-fine': 'manner', 'text': 'How did serfdom develop in and then leave Russia ?'})


# Bước 3: Chuẩn bị dữ liệu, chuyển sang định dạng vector để huấn luyện

In [ ]:
# lấy danh sách các nhãn trong dữ liệu huấn luyện
labels = [x['label-coarse'] for _, x in train]
print(len(labels))
# print(labels[:100])
set_labels = list(set(labels))
print("------")
print("labels:", set_labels)

label2id = {x: i for i, x in enumerate(set_labels)}
print("------")
print(label2id)
id2label = {i: x for i, x in enumerate(set_labels)}
print("------")
print(id2label)

5452
------
labels: ['DESC', 'NUM', 'HUM', 'LOC', 'ENTY', 'ABBR']
------
{'DESC': 0, 'NUM': 1, 'HUM': 2, 'LOC': 3, 'ENTY': 4, 'ABBR': 5}
------
{0: 'DESC', 1: 'NUM', 2: 'HUM', 3: 'LOC', 4: 'ENTY', 5: 'ABBR'}


In [ ]:

label2id['DESC']


0

In [ ]:
id2label[0]

'DESC'

In [ ]:
print("#training size", len(train))
print("#testing size", len(test))
print(train[0])
print(train[1])
print(test[0])
print(test[1])

train_target = [label2id[x['label-coarse']] for _, x in train]
train_data = [x['text'] for _, x in train]


#training size 5452
#testing size 500
(0, {'label-coarse': 'DESC', 'label-fine': 'manner', 'text': 'How did serfdom develop in and then leave Russia ?'})
(1, {'label-coarse': 'ENTY', 'label-fine': 'cremat', 'text': 'What films featured the character Popeye Doyle ?'})
(0, {'label-coarse': 'NUM', 'label-fine': 'dist', 'text': 'How far is it from Denver to Aspen ?'})
(1, {'label-coarse': 'LOC', 'label-fine': 'city', 'text': 'What county is Modesto , California in ?'})


In [ ]:
print(train_data[0], train_target[0])
print(train_data[1], train_target[1])

How did serfdom develop in and then leave Russia ? 0
What films featured the character Popeye Doyle ? 4


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(ngram_range=(1,3))
X_train_counts = count_vect.fit_transform(train_data)
print(X_train_counts.shape)
print(X_train_counts.toarray())
print(count_vect.get_feature_names_out())
from sklearn.feature_extraction.text import TfidfTransformer

# method 1: no-transofrm
X_train = X_train_counts

# method 2: use tf
# tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
# X_train = tf_transformer.transform(X_train_counts)

# method 3: use tf-idf
# tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_counts)
# X_train = tf_transformer.transform(X_train_counts)
# print(X_train.shape)
# print(X_train.toarray())

print("item đầu tiên:", X_train[0].toarray())

(5452, 61819)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
['000' '000 check' '000 for' ... 'zoonose diseases from' 'zorro'
 'zorro ride']
item đầu tiên: [[0 0 0 ... 0 0 0]]


In [ ]:
print(len(train_target))
print(train_target)

5452
[0, 4, 0, 4, 5, 2, 2, 2, 0, 2, 1, 0, 2, 2, 4, 3, 0, 1, 0, 0, 1, 4, 2, 0, 0, 4, 2, 3, 4, 3, 5, 4, 1, 2, 1, 0, 1, 4, 3, 4, 0, 4, 1, 1, 0, 2, 4, 2, 0, 4, 3, 0, 2, 0, 2, 0, 4, 4, 3, 0, 3, 0, 3, 0, 2, 3, 0, 0, 3, 0, 3, 2, 4, 0, 2, 0, 2, 4, 0, 3, 4, 1, 0, 3, 4, 1, 3, 4, 2, 1, 1, 0, 0, 4, 0, 0, 2, 0, 1, 3, 2, 0, 0, 4, 2, 2, 0, 3, 4, 3, 0, 3, 3, 4, 4, 4, 3, 1, 4, 0, 3, 4, 0, 1, 0, 0, 0, 2, 2, 1, 4, 0, 4, 4, 4, 1, 2, 0, 0, 1, 3, 4, 3, 4, 0, 1, 0, 5, 2, 2, 2, 2, 1, 2, 4, 2, 3, 4, 4, 0, 4, 1, 4, 0, 3, 2, 2, 3, 1, 1, 3, 1, 2, 2, 4, 2, 0, 0, 1, 3, 3, 4, 2, 2, 0, 0, 0, 1, 0, 3, 4, 3, 2, 3, 4, 2, 2, 1, 3, 0, 2, 0, 0, 1, 2, 2, 1, 4, 0, 2, 4, 4, 3, 0, 0, 1, 0, 1, 4, 2, 0, 5, 4, 0, 0, 4, 1, 0, 1, 1, 3, 0, 2, 0, 2, 1, 3, 3, 4, 2, 4, 4, 1, 0, 3, 1, 2, 0, 0, 4, 1, 2, 0, 2, 1, 2, 4, 0, 4, 3, 1, 5, 1, 1, 0, 3, 0, 0, 4, 2, 4, 4, 4, 2, 2, 2, 0, 1, 4, 1, 1, 4, 1, 1, 4, 3, 2, 2, 3, 0, 0, 3, 4, 0, 5, 3, 4, 4, 3, 4, 4, 2, 4, 2, 5, 4, 2, 5, 5, 4, 3, 2, 0, 1, 1, 2, 3, 4, 3, 2, 0, 1, 3, 4, 0, 1, 2, 3, 2, 0, 0, 1

# Training X_train_counts

In [ ]:
# method 1
# MultinomialNB
# from sklearn.naive_bayes import MultinomialNB
# clf = MultinomialNB().fit(X_train, train_target)

# method 2
# from sklearn.linear_model import SGDClassifier
# clf = SGDClassifier(loss='hinge', penalty='l2',
#                     alpha=1e-3, random_state=42,
#                     max_iter=100, tol=None)
# clf.fit(X_train, train_target)

# method 3
from sklearn import svm
# https://scikit-learn.org/stable/modules/svm.html#svm
clf = svm.LinearSVC()
clf.fit(X_train, train_target)
print("done !!!")

# method 4
# from sklearn import svm
# clf = svm.SVC(kernel='linear')
# # clf = svm.SVC(kernel='rbf')
# clf.fit(X_train, train_target)

# https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html#sphx-glr-auto-examples-svm-plot-rbf-parameters-py

# method 5

# from sklearn.linear_model import LogisticRegression
# clf = LogisticRegression(penalty='l2', verbose=1)
# clf.fit(X_train, train_target)

done !!!


# Predict

In [ ]:
docs_new = ['what is computer',
            'who is Newton',
            'when is the Tet holiday ?']
X_new_counts = count_vect.transform(docs_new)
X_new = X_new_counts
print("input")
print(X_new.toarray())
# X_new = tf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new)
print("result")
# print([id2label[x] for x in predicted])
for x in predicted:
    print(x, id2label[x])

# for doc, category in zip(docs_new, predicted):
#     print(doc,  category, type(category), id2label[category])

input
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
result
0 DESC
2 HUM
1 NUM


In [ ]:
docs_new = ['who is president of Vietnam',
            'what is the weather today ?']
X_new_counts = count_vect.transform(docs_new)
X_new = X_new_counts
print("input")
print(X_new.toarray())
print(X_new.shape)
# X_new = tf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new)
print("result")
print(predicted)
for x in predicted:
    print(id2label[x])

# for doc, category in zip(docs_new, predicted):
#     print(doc,  category, type(category), id2label[category])

input
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(2, 61819)
result
[2 0]
HUM
DESC


In [ ]:
for i, d in enumerate(docs_new):
    print(i, d, predicted[i], id2label[predicted[i]])

0 who is president of Vietnam 2 HUM
1 what is the weather today ? 0 DESC


# Evaluation

In [ ]:
test_target = [label2id[x['label-coarse']] for _, x in test]
test_data = [x['text'] for _, x in test]
print(len(test_data), len(test_target))
print(test_data[0], test_target[0])


500 500
How far is it from Denver to Aspen ? 1


In [ ]:
X_new_counts = count_vect.transform(test_data)
X_new = X_new_counts
# X_new = tf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new)
print(len(predicted))
print("System")
print(predicted)
print("Gold Label")
print(test_target)

500
System
[1 3 2 0 1 1 2 0 0 0 3 4 1 2 1 1 3 2 0 1 2 0 3 0 0 2 0 4 3 4 1 3 0 1 1 1 3
 1 1 1 4 0 0 0 4 0 1 0 1 2 0 2 2 0 0 0 1 3 3 0 3 2 3 4 3 2 4 0 1 3 1 1 3 2
 3 0 4 1 1 2 0 1 2 1 1 0 0 2 0 3 4 0 1 1 3 3 1 0 0 2 0 2 0 2 3 0 1 4 1 3 4
 0 1 4 0 2 1 4 2 3 1 4 0 2 0 2 1 4 0 1 1 2 0 4 0 0 3 2 2 0 0 4 3 0 0 4 2 4
 0 0 4 2 0 1 2 3 3 0 4 3 0 0 1 3 4 4 1 0 3 2 1 1 1 0 3 3 3 1 4 1 4 0 3 0 0
 2 2 0 3 0 0 3 1 1 0 3 4 2 4 4 4 3 2 4 0 3 2 1 4 0 1 1 3 3 0 4 0 4 1 0 0 1
 0 0 3 4 1 4 3 2 1 2 0 1 4 0 3 0 3 1 4 0 0 0 0 3 0 0 0 0 2 0 4 4 0 3 3 4 0
 4 2 2 1 4 1 0 0 3 1 0 4 0 0 2 0 0 0 1 5 0 3 2 5 0 3 0 0 0 0 0 3 0 1 4 0 0
 4 1 4 4 5 1 0 1 1 3 2 3 2 1 3 3 1 0 3 0 2 3 4 3 0 1 0 4 1 0 0 1 1 0 0 1 4
 4 0 3 0 4 0 1 1 4 1 2 1 2 2 0 1 0 1 3 3 0 0 2 1 0 0 4 0 0 4 1 0 0 4 4 0 3
 1 4 0 0 2 1 3 0 5 0 2 2 4 2 1 1 0 2 0 0 2 0 0 1 0 0 0 1 1 3 0 0 1 5 1 3 0
 3 1 0 2 4 3 5 3 0 0 0 2 2 1 0 4 0 3 4 0 3 0 2 0 4 3 2 0 0 0 1 5 4 2 4 3 4
 2 4 0 2 1 4 4 4 0 1 0 1 4 3 0 0 4 0 0 0 3 4 0 4 0 0 0 0 4 1 4 4 0 3 4 3 4
 4 4 0 3 0 3 1

In [ ]:
ncorrect = sum([y_pred == y for y_pred, y in zip (predicted, test_target)])
ncorrect

443

In [ ]:
count = 0
for i, system_label in enumerate(predicted):
    gold_label = test_target[i]
#     print(system_label, gold_label)
    if system_label == gold_label:
        count += 1
print(count)

443


In [ ]:
accurracy = ncorrect / len(test_target)
print("accurracy", accurracy)

accurracy 0.886


In [ ]:
# Check the vector of each sample

In [ ]:
train_data[0]

'How did serfdom develop in and then leave Russia ?'

In [ ]:
for i in X_train[0]:
    print(i)

  (0, 2355)	1
  (0, 2904)	1
  (0, 2906)	1
  (0, 12645)	1
  (0, 12646)	1
  (0, 12647)	1
  (0, 12765)	1
  (0, 13144)	1
  (0, 13145)	1
  (0, 22825)	1
  (0, 22860)	1
  (0, 22873)	1
  (0, 23671)	1
  (0, 23821)	1
  (0, 23823)	1
  (0, 29005)	1
  (0, 29010)	1
  (0, 42229)	1
  (0, 43147)	1
  (0, 43148)	1
  (0, 43149)	1
  (0, 51584)	1
  (0, 51593)	1
  (0, 51594)	1


In [ ]:
print(X_train[0].toarray())

[[0 0 0 ... 0 0 0]]


In [ ]:
for i, x in enumerate(X_train[0].toarray()[0]):
    if x > 0:
        print(i, count_vect.get_feature_names_out()[i])

2355 and
2904 and then
2906 and then leave
12645 develop
12646 develop in
12647 develop in and
12765 did
13144 did serfdom
13145 did serfdom develop
22825 how
22860 how did
22873 how did serfdom
23671 in
23821 in and
23823 in and then
29005 leave
29010 leave russia
42229 russia
43147 serfdom
43148 serfdom develop
43149 serfdom develop in
51584 then
51593 then leave
51594 then leave russia


In [ ]:
len(count_vect.get_feature_names_out())

61819